In [1]:
import tvm
import numpy as np
from tvm import relay
from tvm import autotvm
import tvm.contrib.graph_runtime as runtime

In [2]:
x = relay.var('x', shape=[1, 32, 32, 32], dtype='int16')
w = relay.var('w', shape=[32, 32, 3, 3], dtype='int16')

y = relay.nn.conv2d(x, w, kernel_size=[3,3], data_layout='NCHW', binarize=True, out_dtype='int16')
y_func = relay.Function([x, w], y)

In [3]:
w_np = np.random.normal(size=[32, 32, 3, 3]).astype('int16')
params = {'w': w_np}
with relay.build_config(opt_level=0):
    graph, lib, params = relay.build_module.build(y_func, target='llvm', params=params)

Cannot find config for target=llvm, workload=('bitserial_conv2d_nchw', (1, 32, 32, 32, 'int16'), (32, 32, 3, 3, 'int16'), (1, 1), (0, 0), 1, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.


In [4]:
module = runtime.create(graph, lib, tvm.context('llvm', 0))
module.set_input('x', np.random.uniform(size=(1, 32, 32, 32)).astype('int16'))
module.set_input(**params)

In [5]:
module.run()
output = module.get_output(0)